## Twitter_Project

### Importing modules

In [1]:
import pandas as pd
import numpy as np 
import wordcloud
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import re
import string
import time
import emoji

### Data Preprocessing and analysis

In [2]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv',names=['Sentiments','ID','Time','query','User','text'],encoding_errors='ignore')

In [3]:
df.tail()

,Sentiments,ID,Time,query,User,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Sentiments  1600000 non-null  int64 
 1   ID          1600000 non-null  int64 
 2   Time        1600000 non-null  object
 3   query       1600000 non-null  object
 4   User        1600000 non-null  object
 5   text        1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [5]:
df.drop('query',inplace=True,axis=1)

#### Removing tags and storing them

In [6]:
def find_tags(str1):
    temp = re.findall('@[a-zA-Z0-9._-]+\s', str1)
    ans=[]
    for i in temp:
        ans.append(i[:-1])
    return ans

In [7]:
df['tags'] = df['text'].apply(find_tags)

In [8]:
def remove_tags(str1):
    return re.sub('@[a-zA-Z0-9._-]+\s', '',str1)

In [9]:
df['text']  =df['text'].apply(remove_tags)

In [10]:
df['text']

0          http://twitpic.com/2y1zl - Awww, that's a bumm...
1          is upset that he can't update his Facebook by ...
2          I dived many times for the ball. Managed to sa...
3            my whole body feels itchy and like its on fire 
4          no, it's not behaving at all. i'm mad. why am ...
                                 ...                        
1599995    Just woke up. Having no school is the best fee...
1599996    TheWDB.com - Very cool to hear old Walt interv...
1599997    Are you ready for your MoJo Makeover? Ask me f...
1599998    Happy 38th Birthday to my boo of alll time!!! ...
1599999                               happy #charitytuesday 
Name: text, Length: 1600000, dtype: object

In [11]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i had",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

### Removing Contractions and lowercasing

In [12]:
def contract(text):
    text = text.lower()
    for i in contractions.keys():
        if i in text:
            text = re.sub(i,contractions[i],text)
    return text

In [13]:
df['text'] = df['text'].apply(contract)

In [14]:
df['text']

0          http://twitpic.com/2y1zl - awww, that is a bum...
1          is upset that he cannot update his facebook by...
2          i dived many times for the ball. managed to sa...
3            my whole body feels itchy and like its on fire 
4          no, it is not behaving at all. i am mad. why a...
                                 ...                        
1599995    just woke up. having no school is the best fee...
1599996    thewdb.com - very cool to hear old walt interv...
1599997    are you ready for your mojo makeover? ask me f...
1599998    happy 38th birthday to my boo of alll time!!! ...
1599999                               happy #charitytuesday 
Name: text, Length: 1600000, dtype: object

### Removing punctuations

In [15]:
def remove_punc(text):
    temp = [t for t in text if t not in string.punctuation]
    return "".join(temp)

In [16]:
df['text'] = df['text'].apply(remove_punc)

In [17]:
df.head()

,Sentiments,ID,Time,User,text,tags
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,httptwitpiccom2y1zl awww that is a bummer yo...,[@switchfoot]
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he cannot update his facebook by...,[]
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,i dived many times for the ball managed to sav...,[@Kenichan]
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire,[]
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,no it is not behaving at all i am mad why am i...,[@nationwideclass]


### Removing unnecessary info

In [18]:
df.drop(['ID','Time','User'],inplace=True,axis=1)

In [19]:
df.head()

,Sentiments,text,tags
0,0,httptwitpiccom2y1zl awww that is a bummer yo...,[@switchfoot]
1,0,is upset that he cannot update his facebook by...,[]
2,0,i dived many times for the ball managed to sav...,[@Kenichan]
3,0,my whole body feels itchy and like its on fire,[]
4,0,no it is not behaving at all i am mad why am i...,[@nationwideclass]


In [20]:
df['tags']  = df['tags'].apply(len)

### Removing extra white spaces

In [21]:
def remove_white (text):
    return " ".join(text.split())

In [22]:
df['text'] = df['text'].apply(remove_white)

In [23]:
df.head()

,Sentiments,text,tags
0,0,httptwitpiccom2y1zl awww that is a bummer you ...,1
1,0,is upset that he cannot update his facebook by...,0
2,0,i dived many times for the ball managed to sav...,1
3,0,my whole body feels itchy and like its on fire,0
4,0,no it is not behaving at all i am mad why am i...,1


### Getting text length

In [24]:
df.tail(1)['text'] = "happy charity tuesday"

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [25]:
df['text_length']  = df['text'].apply(lambda x  : len(x.split()))

In [26]:
df.head()

,Sentiments,text,tags,text_length
0,0,httptwitpiccom2y1zl awww that is a bummer you ...,1,18
1,0,is upset that he cannot update his facebook by...,0,21
2,0,i dived many times for the ball managed to sav...,1,17
3,0,my whole body feels itchy and like its on fire,0,10
4,0,no it is not behaving at all i am mad why am i...,1,22


### Average word length

In [27]:
def word_length(text):
    c=0
    for i in text.split():
        c+=len(i)
    return c/(len(text.split())+1)

In [28]:
df['word_length']  = df['text'].apply(word_length)

### Creating a dictionary mapping every unique word to its count

In [29]:
word_count = {}

In [30]:
temp = " ".join(df['text'])

In [31]:
for i in temp.split():
    if i in word_count:
        word_count[i] +=1
    else:
        word_count[i] = 1

In [32]:
word_count

{'httptwitpiccom2y1zl': 1,
 'awww': 5272,
 'that': 162941,
 'is': 362841,
 'a': 377500,
 'bummer': 1442,
 'you': 298187,
 'shoulda': 348,
 'got': 60749,
 'david': 2279,
 'carr': 75,
 'of': 183704,
 'third': 816,
 'day': 82365,
 'to': 564506,
 'do': 114788,
 'it': 298244,
 'd': 5452,
 'upset': 2669,
 'he': 37507,
 'cannot': 47789,
 'update': 4257,
 'his': 15392,
 'facebook': 4215,
 'by': 25244,
 'texting': 729,
 'and': 298309,
 'might': 9638,
 'cry': 4408,
 'as': 38973,
 'result': 611,
 'school': 19709,
 'today': 64601,
 'also': 10265,
 'blah': 1464,
 'i': 931377,
 'dived': 3,
 'many': 8819,
 'times': 7652,
 'for': 215677,
 'the': 520295,
 'ball': 1263,
 'managed': 978,
 'save': 2193,
 '50': 1275,
 'rest': 5215,
 'go': 72910,
 'out': 80516,
 'bounds': 19,
 'my': 314026,
 'whole': 5960,
 'body': 2480,
 'feels': 5132,
 'itchy': 446,
 'like': 77745,
 'its': 43321,
 'on': 167073,
 'fire': 1503,
 'no': 73511,
 'not': 234670,
 'behaving': 68,
 'at': 111277,
 'all': 81903,
 'am': 182027,
 'mad

### Removing words with low count

In [33]:
df['text_split'] = df['text'].apply(lambda x  : x.split())

In [34]:
def low_count_remover(text):
    temp=[]
    for i in text:
        if word_count[i] !=1:
            temp.append(i)
    return temp

In [35]:
df['text_split']  = df['text_split'].apply(low_count_remover)

In [36]:
df['text_split']

0          [awww, that, is, a, bummer, you, shoulda, got,...
1          [is, upset, that, he, cannot, update, his, fac...
2          [i, dived, many, times, for, the, ball, manage...
3          [my, whole, body, feels, itchy, and, like, its...
4          [no, it, is, not, behaving, at, all, i, am, ma...
                                 ...                        
1599995    [just, woke, up, having, no, school, is, the, ...
1599996     [very, cool, to, hear, old, walt, interviews, ♫]
1599997    [are, you, ready, for, your, mojo, makeover, a...
1599998    [happy, 38th, birthday, to, my, boo, of, alll,...
1599999                            [happy, charity, tuesday]
Name: text_split, Length: 1600000, dtype: object

### DeEmojify text

In [37]:
def deEmojify(text):
    temp=[]
    for i in text:
        temp.append(i.encode('ascii', 'ignore').decode('ascii'))
    temp1=[j for j in temp if len(j)!=0]
    return temp1

In [38]:
df['text_split'] = df['text_split'].apply(deEmojify)

### Lemmatization

In [39]:
from textblob import Word

In [40]:
def lemma(text):
    temp=[]
    for i in text:
        u = Word(i)
        temp.append(u.lemmatize())
    return temp

In [41]:
%%time
df['text_split'] = df['text_split'].apply(lemma)

Wall time: 1min 28s


### Dictionary mapping each word to a unique integer

In [42]:
from nltk.corpus import stopwords

In [43]:
word2int = {}

In [44]:
temp = []
for i in df['text_split']:
    temp.extend(i)

In [45]:
cr = 0
for j in temp:
    if j not in word2int:
        word2int[j] = cr
        cr+=1

In [46]:
df['text2int']  = df['text_split'].apply(lambda x : [word2int[t] for t in x ])

In [47]:
df['text_length'] = df['text2int'].apply(lambda x: len(x))

### Reverse Dictionary

In [48]:
int2word ={}
for i,j in word2int.items():
    int2word[j] = i

### Padding and forming X and y 

In [49]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [50]:
X = []
for i in df['text2int']:
    X.append(i)

In [51]:
X = pad_sequences(X , maxlen=120,padding='post',truncating='post')

In [52]:
y = df['Sentiments']

In [53]:
def converter(x):
    if x==4:
        return 1
    else:
        return 0

In [54]:
y = y.apply(converter)

### Embedding

#### Glove Vectors

In [55]:
file = open('glove.twitter.27B.200d.txt' ,encoding='utf-8')

In [56]:
glove_vectors={}
for line in file:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:])
    glove_vectors[word] = vector
file.close()

In [57]:
## Getting vectors for our vocab

In [58]:
%%time
word_vector = np.zeros((len(word2int),200))
for word,index in word2int.items():
    if glove_vectors.get(word) is not None:
        word_vector[index] = np.asarray(glove_vectors.get(word))

Wall time: 15.4 s


### Creating Model

In [59]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv1D,GlobalAvgPool1D,Embedding,MaxPool1D,LSTM

In [67]:
model = Sequential()
model.add(Embedding(len(word2int) , 200 ,input_length = 120 , weights = [word_vector] ,trainable=False  ))
model.add(Conv1D(128,8,activation='relu'))
model.add(MaxPool1D(2))
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16,activation='relu'))
model.add(GlobalAvgPool1D())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'])

In [68]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=7)

Epoch 1/7
35000/35000 [==============================] - 202s 6ms/step - loss: 0.4882 - accuracy: 0.7710 - val_loss: 0.5171 - val_accuracy: 0.7397
Epoch 2/7
35000/35000 [==============================] - 202s 6ms/step - loss: 0.4572 - accuracy: 0.7903 - val_loss: 0.5112 - val_accuracy: 0.7540
Epoch 3/7
35000/35000 [==============================] - 199s 6ms/step - loss: 0.4467 - accuracy: 0.7965 - val_loss: 0.4910 - val_accuracy: 0.7896
Epoch 4/7
35000/35000 [==============================] - 195s 6ms/step - loss: 0.4396 - accuracy: 0.8005 - val_loss: 0.4897 - val_accuracy: 0.7809
Epoch 5/7
35000/35000 [==============================] - 195s 6ms/step - loss: 0.4349 - accuracy: 0.8029 - val_loss: 0.4831 - val_accuracy: 0.7874
Epoch 6/7
35000/35000 [==============================] - 231s 7ms/step - loss: 0.4311 - accuracy: 0.8054 - val_loss: 0.4837 - val_accuracy: 0.7831
Epoch 7/7
35000/35000 [==============================] - 196s 6ms/step - loss: 0.4274 - accuracy: 0.8074 - val_loss: 0

In [69]:
def Sentiment(text):
    text = remove_tags(text)
    text = contract(text)
    text = remove_punc(text)
    text = remove_white(text)
    text = text.split()
    text = deEmojify(text)
    text = lemma(text)
    temp=[]
    for i in text:
        if i in word2int:
            temp.append(word2int[i])
    while len(temp)!=120:
        temp.append(0)
    if model.predict_classes([temp]) ==[[1]]:
        print(emoji.emojize("\U0001f970"))
    else:
        print('\U0001f97a')

In [73]:
Sentiment("You're welcome")

🥰
